In [1]:
# inport BeautifulSoup
import re
from bs4 import BeautifulSoup

In [2]:
folder = "20170603"
file=open(folder + "/a.html", "r")
soup = BeautifulSoup(file.read(), "html.parser")
file.close()

In [3]:
#print(soup.find_all("div","playerInfo"))

In [4]:
# name, age, TSI, pay per week, form, health(체력),
column = 'name, age, TSI, pay per week, form, health(체력)'
f = open(folder + '/player.txt', 'a', encoding='utf-8')
f.write(column +'\n')
f.close()

In [5]:
for div in soup.find_all("div","playerInfo"):
    # get String
    line = div.get_text()
    
    # parsing String
    line = line.replace(':\n                \n\n ', ':')
    line = line.replace('\n\n\n', '')
    line = line.replace('\n\n                        1', '\n1')
    line = line.replace('\n\n                        2', '\n2')
    line = line.replace('\n\n                        3', '\n3')
    line = line.replace('TSI = ','\nTSI:')
    line = line.replace('\xa0', '')
    line = line.replace(', 임금 = ', '\n임금:')
    line = line.replace('Won', ' Won\n')
    line = line.replace('form, ', 'form\n')
    if line.find('체력 [') > -1 :
        line = line.replace('체력 [', '체력\n특수능력:')
        line = line.replace("예측할 수 없음", "예측못함")
        line = line.replace(']', '\n')
    else :
        line = line.replace('체력', '체력\n특수능력:\n')
    line = line.replace('경험과 ', '경험\n')
    line = line.replace('리더십 능력', '리더십능력\n')
    line = line.replace('\n\n', '\n')
    line = line.replace('\n                    ', '')
    line = line.replace('충성도 ', '충성도:')
    line = line.replace('골키핑', '\n골키핑')
    line = line.replace('수비력','\n수비력')
    line = line.replace('플레이메이킹', '\n플레이메이킹')
    line = line.replace('윙어', '\n윙어')
    line = line.replace('패스력','\n패스력')
    line = line.replace('득점력', '\n득점력')
    line = line.replace('세트 피스','\n세트피스')
    
    line = line.replace('신의 경지','20')
    line = line.replace('초월적', '19')
    line = line.replace('마술적', '18')
    line = line.replace('전설적', '17')
    line = line.replace('외계인', '16')
    line = line.replace('막강함', '15')
    line = line.replace('초자연적', '14')
    line = line.replace('세계 최고', '13')
    line = line.replace('훌륭함', '12')
    line = line.replace('눈부심', '11')
    line = line.replace('걸출함', '10')
    line = line.replace('강력함', '9')
    line = line.replace('뛰어남', '8')
    line = line.replace('우수', '7')
    line = line.replace('무난함', '6')
    line = line.replace('불충분함', '5')
    line = line.replace('약함', '4')
    line = line.replace('나쁨', '3')
    line = line.replace('형편 없음', '2')
    line = line.replace('최악', '1')
    line = line.replace('없음', '0')
    
    line = line.replace('(\n', '(')
    line = line.replace('\n (', ':')
    line = line.replace(')', '')
    
    line = line.replace('Je-Min Jung    ', 'Je-Min Jung\n')
    line = line.replace('\n', ',')
    
    # name, age, TSI, pay per week, form, health(체력),
    
    line = line.replace(' 세', '세')
    line = line.replace(' 일', '일')
    line = line.replace('TSI:', '')
    line = line.replace('임금:주급 ', '')
    line = line.replace('0 Won','0')
    line = line.replace(' form', '')
    line = line.replace(' 체력', '')
    line = line.replace('특수능력:', '')
    line = line.replace(' 경험', '')
    line = line.replace(' 리더십능력', '')
    
    line = re.sub(r'[0-9]{4}-[0-9]{2}-[0-9]{2}:.*,', "", line)
    
    line.encode('utf-8')
    f = open(folder + '/player.txt', 'a', encoding='utf-8')
       
    for lines in line.split('\n'):
        if (re.match('^[0-9]{1,2}\. ', lines)) :
            lines = re.sub(r',$', "", lines)
            print(lines)
            f.write(lines +'\n')
    f.close()

1. Jee-Seung Oh,21세 52일,2430,990000,5,6,,3,4,충성도:20,골키핑:6,수비력:3,플레이메이킹:1,윙어:1,패스력:1,득점력:1,세트피스:6
2. Myung-Jin Min,27세 103일,2240,930000,5,7,,4,1,충성도:20,골키핑:6,수비력:4,플레이메이킹:1,윙어:1,패스력:2,득점력:1,세트피스:2
3. Eun-Seok Hong,28세 86일,560,300000,6,6,,4,5,충성도:20,골키핑:1,수비력:6,플레이메이킹:6,윙어:3,패스력:2,득점력:2,세트피스:4
4. Ji-Woon Lee,18세 102일,870,390000,4,7,,2,5,충성도:20,골키핑:1,수비력:7,플레이메이킹:4,윙어:3,패스력:5,득점력:2,세트피스:5
5. Jang-Soo Choi,30세 36일,170,280000,3,6,빠름,5,5,충성도:20,골키핑:1,수비력:6,플레이메이킹:4,윙어:4,패스력:2,득점력:1,세트피스:7
6. Jae-Shin Cha,26세 83일,1140,390000,6,6,,4,5,충성도:20,골키핑:1,수비력:7,플레이메이킹:3,윙어:6,패스력:3,득점력:2,세트피스:6
7. In-Moo Byon,31세 71일,550,300000,5,6,,5,1,충성도:20,골키핑:2,수비력:6,플레이메이킹:5,윙어:5,패스력:5,득점력:3,세트피스:6
8. Hong-Chan Son,21세 31일,1010,310000,6,6,,3,5,충성도:20,골키핑:1,수비력:5,플레이메이킹:6,윙어:5,패스력:5,득점력:1,세트피스:4
9. Hoi-Taek Park,19세 38일,720,330000,3,5,예측못함,2,2,충성도:20,골키핑:1,수비력:5,플레이메이킹:6,윙어:3,패스력:6,득점력:3,세트피스:6
10. Kwang-Eul Jeon,23세 37일,1350,430000,6,6,,3,4,충성도:20,골키핑:1,수비력:6,플레이메이킹:7,윙어:3,패스력:3,득점력:2,세트피스:5
11. Kwang-Hwan Yun,26

In [6]:
import psycopg2

In [7]:
try:
    # use our connection values to establish a connection
    connect_str = "dbname='mydatabase' user='myuser' host='localhost' port='65432' password='123qwe' "
    conn = psycopg2.connect(connect_str)
    
except Exception as e:
    print("Uh oh, can't connect. Invalid dbname, user or password?")
    print(e)

In [8]:
# create a psycopg2 cursor that can execute queries
cursor = conn.cursor()

In [9]:
# create a new table with a single column called "name"
cursor.execute("""CREATE TABLE tutorials (name char(40)) ;""")

In [10]:
# insert a row to table
cursor.execute("""INSERT INTO tutorials (name) VALUES ('testName') ;""")

In [11]:
# run a SELECT statement - no data in there, but we can try it
cursor.execute("""SELECT * from tutorials ;""")
rows = cursor.fetchall()
print(rows)

[('testName                                ',)]
